Use REBOUND commit 25f856dc2f79e0ad17b2f6bd604225f550593376

The runs use REBOUND commit a675e6f92cba32bae86f94739faec3b1b3098e81. The problem is that this version doesn't have a __del__ function for the simulation archives and you get an error of too many open files when you run this. Instead use the current latest commit (25f856dc2f79e0ad17b2f6bd604225f550593376) which does. I test below that I can reproduce the integrations bit by bit, so this should work.

Set path below to location of stabilitydataset folder

In [1]:
import rebound
rebound.__version__

'3.8.1'

In [2]:
if rebound.__githash__ != '25f856dc2f79e0ad17b2f6bd604225f550593376':
    print('Should checkout commit above to ensure this runs correctly')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

path = '/mnt/ssd/workspace/stability/stabilitydataset/' # path to stabilitydataset folder

In [4]:
pathtosa = path+'data/resonant/simulation_archives/runs/'
pathtossa = path+'data/resonant/simulation_archives/shadowruns/'
df = pd.read_csv(path+'data/resonant/runstrings.csv', index_col=0)
df.tail()

,runstring
9995,0009995.bin
9996,0009996.bin
9997,0009997.bin
9998,0009998.bin
9999,0009999.bin


In [5]:
df['instability_time'] = -1
df['shadow_instability_time'] = -1
df['Stable'] = -1
df.head()

,runstring,instability_time,shadow_instability_time,Stable
0,0000000.bin,-1,-1,-1
1,0000001.bin,-1,-1,-1
2,0000002.bin,-1,-1,-1
3,0000003.bin,-1,-1,-1
4,0000004.bin,-1,-1,-1


In [6]:
def labels(row):
    try:
        sa = rebound.SimulationArchive(pathtosa+'sa'+row['runstring'])
        sim = sa[-1]
        row['instability_time'] = sim.t
        ssa = rebound.SimulationArchive(pathtossa+'sa'+row['runstring'])
        ssim = ssa[-1]
        row['shadow_instability_time'] = ssim.t
        row['Stable'] = row['instability_time'] > 9.99e8
    except:
        print(row['runstring'])
    return row

In [7]:
%%time
df = df.apply(labels, axis=1)

/mnt/ssd/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)


0002058.bin
CPU times: user 11.8 s, sys: 12.9 s, total: 24.7 s
Wall time: 24.7 s


In [8]:
df.head()

,runstring,instability_time,shadow_instability_time,Stable
0,0000000.bin,1.545872e+06,3.063700e+06,False
1,0000001.bin,9.990000e+08,9.990000e+08,True
2,0000002.bin,9.990000e+08,9.990000e+08,True
3,0000003.bin,2.287671e+06,8.392234e+06,False
4,0000004.bin,9.668931e+05,3.380350e+05,False


In [9]:
df.to_csv(path+'data/resonant/labels.csv', encoding='ascii')

In [10]:
pathtosa = path+'data/resonant/simulation_archives/runs/'
pathtossa = path+'data/resonant/simulation_archives/shadowruns/'
df = pd.read_csv(path+'data/resonant/runstrings.csv', index_col=0)
df.tail()

,runstring
9995,0009995.bin
9996,0009996.bin
9997,0009997.bin
9998,0009998.bin
9999,0009999.bin


In [11]:
df['m1'] = -1
df['m2'] = -1
df['m3'] = -1
df.head()

,runstring,m1,m2,m3
0,0000000.bin,-1,-1,-1
1,0000001.bin,-1,-1,-1
2,0000002.bin,-1,-1,-1
3,0000003.bin,-1,-1,-1
4,0000004.bin,-1,-1,-1


In [12]:
def masses(row):
    try:
        sa = rebound.SimulationArchive(pathtosa+'sa'+row['runstring'])
        sim = sa[0]
        row['m1'] = sim.particles[1].m
        row['m2'] = sim.particles[2].m
        row['m3'] = sim.particles[3].m
    except:
        print(row['runstring'])
    return row

In [13]:
%%time
df = df.apply(masses, axis=1)

0002058.bin
CPU times: user 6.3 s, sys: 6.54 s, total: 12.8 s
Wall time: 12.8 s


In [14]:
df.head()

,runstring,m1,m2,m3
0,0000000.bin,-1,-1,-1
1,0000001.bin,-1,-1,-1
2,0000002.bin,-1,-1,-1
3,0000003.bin,-1,-1,-1
4,0000004.bin,-1,-1,-1


In [15]:
df.to_csv(path+'data/resonant/masses.csv', encoding='ascii')